## Susceptible depletion
Building on the previous (`01basics`) notebook,
we'll now start to think about susceptible depletion.

Again, from this equation in Faria, et al:
$\\i_{s,t} = (1-\frac{n_{s,t}}{N})R_{s,t}\sum_{\tau<t} i_{s,\tau}g_{t-\tau}$

And again reducing the complexity of this by ignoring strains,
we'll now consider the equation with susceptible depletion included:
$\\i_t = (1-\frac{n_t}{N})R_t\sum_{\tau<t} i_{\tau}g_{t-\tau}$

In [ ]:
from typing import Dict
from scipy.stats import gamma
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'

### Parameters
Set model parameters, now including the population size.
Also get the generation times as described previously.

In [ ]:
n_times = 40
seed = 1.0
r0 = 2.0
pop = 100.0
incidence = np.zeros(n_times)
incidence[0] = seed
gen_time_sd = 1.5
gen_time_mean = 5.0

def get_gamma_params_from_mean_sd(req_mean: float, req_sd: float) -> Dict[str, float]:
    var = req_sd ** 2.0
    scale = var / req_mean
    a = req_mean / scale
    return {'a': a, 'scale': scale}

gamma_params = get_gamma_params_from_mean_sd(gen_time_mean, gen_time_sd)
gen_time_densities = np.diff(gamma.cdf(range(n_times + 1), **gamma_params))

### Model run
Run the model with susceptible depletion,
decrementing the susceptible population by the incidence at each step.
We'll also zero out any negative values for the susceptibles
that could occur if the time step is too large.
For reasonable parameter values, these typically seem to be very small.

In [ ]:
suscept = pop - seed
for t in range(1, n_times):
    suscept_prop = suscept / pop
    infect_contribution_by_day = incidence[:t] * gen_time_densities[:t][::-1] * r0
    this_inc = infect_contribution_by_day.sum() * suscept_prop
    incidence[t] = this_inc
    suscept = max(suscept - this_inc, 0.0)
pd.Series(incidence).plot(labels={'index': 'day', 'value': 'incidence'})

Now with susceptible depletion, we have an epi-curve that goes up in the initial phase with $R_0 > 1$,
but comes back down as susceptibles are depleted and so $R_t$ falls below one.